In [6]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [10]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='02-experiment-tracking-homework', tags={}>]

In [11]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=["metrics.rmse ASC"]
)

In [12]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f16b59b20c8342aa9592fde07976af7d, rmse: 6.2961
run id: 7a46c9cd4dde45f2a1c3919a0e97dc2d, rmse: 6.2961
run id: cd0a40c6bf8c4f72ac6bf73f95638b9d, rmse: 6.2963
run id: 7fef5a50ddd04b4d86a9ea2c50037da5, rmse: 6.3027
run id: 97eef540368b4353a8c29596db00e7fe, rmse: 6.3071
run id: 2c09a4fe15134e1f875a8ecfe4c0df1c, rmse: 6.3111
run id: f046b36358ee4a0d862f8188c814d4f6, rmse: 6.3250
run id: 45d645163f8f409f95a7638144fbc7e4, rmse: 6.3292
run id: 9f124247883641f6afc013a858ac91bc, rmse: 6.3519
run id: 47271a05428a4959bd51fd937d6c1541, rmse: 6.3560


In [13]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [14]:
run_id = "f539394805074f4296c1ed6b0a1d905d"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/29 13:51:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 4
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1653832287847, current_stage='None', description=None, last_updated_timestamp=1653832287847, name='nyc-taxi-regressor', run_id='f539394805074f4296c1ed6b0a1d905d', run_link=None, source='./mlruns/1/f539394805074f4296c1ed6b0a1d905d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [15]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1653760373223, description='', last_updated_timestamp=1653832287847, latest_versions=[<ModelVersion: creation_timestamp=1653813055853, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2022-05-29', last_updated_timestamp=1653813697602, name='nyc-taxi-regressor', run_id='f539394805074f4296c1ed6b0a1d905d', run_link=None, source='./mlruns/1/f539394805074f4296c1ed6b0a1d905d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>,
  <ModelVersion: creation_timestamp=1653832287847, current_stage='None', description=None, last_updated_timestamp=1653832287847, name='nyc-taxi-regressor', run_id='f539394805074f4296c1ed6b0a1d905d', run_link=None, source='./mlruns/1/f539394805074f4296c1ed6b0a1d905d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>], name='nyc-taxi-regressor', tags={}>]

In [17]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging
version: 4, stage: None


In [18]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653813055853, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2022-05-29', last_updated_timestamp=1653832312044, name='nyc-taxi-regressor', run_id='f539394805074f4296c1ed6b0a1d905d', run_link=None, source='./mlruns/1/f539394805074f4296c1ed6b0a1d905d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [19]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1653813055853, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2022-05-29', last_updated_timestamp=1653832315026, name='nyc-taxi-regressor', run_id='f539394805074f4296c1ed6b0a1d905d', run_link=None, source='./mlruns/1/f539394805074f4296c1ed6b0a1d905d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [20]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [21]:
df = read_dataframe("../data/green_tripdata_2021-03.parquet")

In [23]:
run_id = "f16b59b20c8342aa9592fde07976af7d"

client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/azureuser/notebooks/02-experiment-tracking/preprocessor'

In [24]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [25]:
X_test = preprocess(df, dv)

In [26]:
target = "duration"
y_test = df[target].values

In [27]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[13:55:35] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 11.5 s, sys: 60.5 ms, total: 11.6 s
Wall time: 8.32 s


{'rmse': 6.243147412562448}

In [28]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

[13:55:51] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 10.9 s, sys: 3.2 ms, total: 10.9 s
Wall time: 5.48 s


{'rmse': 6.243147412562448}

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)